In [1]:
from numpy import pi, linspace, diff, mean, min, max, arctan2
from skyfield.api import load
from skyfield.framelib import ecliptic_J2000_frame, ICRS, mean_equator_and_equinox_of_date
from skyfield.searchlib import find_discrete
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
ts = load.timescale()
eph_s_name = 'de421.bsp'
# eph_s_name = 'de422.bsp'
t_start = ts.tt(2005, 6, 21, 0, 0, 0)
eph_s = load(eph_s_name)
moon = eph_s["moon"]
earth = eph_s["earth"]
sun = eph_s["sun"]
northern_hemisphere = True

## Max lunar declination

In [4]:
def f_max_dec(target, ref):
    def zero(t):
        _, _, _, lat_r, _, _  = ref.at(t).observe(target).frame_latlon_and_rates(ICRS)
        return  lat_r.degrees.per_day > 0
    zero.step_days = 10.0
    return zero

In [5]:
t_dec_zero_rate = find_discrete(t_start, t_start + 365*19.5, f_max_dec(moon, earth))
t_max_dec = t_dec_zero_rate[0][t_dec_zero_rate[1] == int(not northern_hemisphere)]
ra, dec, _ = earth.at(t_max_dec).observe(moon).radec()

In [6]:
colors = [f"#{255:02x}{int(255 * (t_max_dec.tt[-1] - t) / (t_max_dec.tt[-1] - t_max_dec.tt[0])):02x}{255:02x}" for t in t_max_dec.tt]
fig = figure(title="Moon standstills", x_axis_label="RA(hours)", y_axis_label="Dec(deg)", width=1100)
fig.scatter(ra.hours, dec.degrees, fill_color=colors, line_color="red", size=7)
fig.line(ra.hours, dec.degrees, line_color="blue")
show(fig)